This code will create a STAC catalog and add several items from a local folder

In [1]:
#Install python libraries used in this script
%pip install rasterio shapely pystac piexif pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use update

In [2]:
########Import libraries, modules, and classes into environment. ########
###########################

#Import the Path class from the pathlib module in Python. \
#The pathlib module is part of the Python Standard Library and provides \
#an object-oriented way to work with file system paths, \
#making it more convenient and readable than using traditional string-based file paths.

#The Path class is a high-level and flexible representation of file system paths. \
#It can be used for various purposes, such as constructing file system paths, navigating directories, \
#creating directories, reading or writing files, and more.
from pathlib import Path

#The os module is part of the Python Standard Library and provides a \
#collection of functions for interacting with the operating system. 
#This module allows you to perform various tasks such as creating, deleting, or modifying files and directories, \
#obtaining system information, and managing processes.
import os


#Import the pystac library into environment. Pystac (https://pystac.readthedocs.io/en/stable/index.html)
#is a library for reading and writing STAC stuff
import pystac 

#ProjectionExtension is a class within the module 'pystac.extensions.projection'. \
#We are using it to show the map projection of raster dataset
from pystac.extensions.projection import ProjectionExtension

#RasterExtension is a class within the module 'pystac.extensions.raster'
#we are using it to display various info about the raster
from pystac.extensions.raster import RasterExtension


from pystac.extensions.scientific import ScientificExtension
from pystac.extensions.scientific import Publication

from pystac.extensions.raster import RasterExtension


from pystac.provider import Provider
from pystac.provider import ProviderRole
import pystac.provider 
from pystac import Provider


#rasterio is a library to read and write raster data. It will be used in this script to extract information \
#from drone imagery geotiffs 
import rasterio
import rasterio.warp #used to convert coordinate of a raster bounding box
#import rasterio.features


#Used to create a polygon of the raster bounds
from shapely.geometry import Polygon, mapping

#For creating a geojson output file of STAC items. This module comes in the Python Standard Library
import json


# datetime is a module in the Python's standard library. We need it to assign collection dates to each item or asset
from datetime import datetime

#We will use pandas to bring a csv file into a dataframe
import pandas


In [10]:
######This cell is for users to input manual metadata for imagery assets that will become STAC catalogs.#######
############################

# source images folder
#source_image_folder = '/data-store/iplant/home/jgillan/STAC_drone'
source_image_folder = '/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019'


# output folder (will overwrite existing files with matching output names)
stac_output_directory = '/data-store/iplant/home/jgillan/STAC_drone/SRER_May2019'
#stac_output_directory = 'https://data.cyverse.org/dav/iplant/home/jgillan/STAC_drone/stac_may10'


# Additional Metadata to add to items
platform = 'DJI Phantom 4 RTK'
license = 'CC-BY-SA-4.0'
items_mission_description = 'The imagery was part of the Ecostate Mapping project of 2019 at Santa Rita Experimental Range'
pub_doi = '10.1002/ecs2.3649'
citation = 'Gillan, JK., GE Ponce-Campos, TL Swetnam, A Gorlier, P Heilman, MP McClaran. 2021. Innovations to expand drone data collection and analysis for rangeland monitoring. Ecosphere, 12(7)'

# collection definitions
collection_id = 'Santa Rita Ecostate Mapping - May 2019' # needs to be folder-name compatible
collection_description = 'The imagery was part of the Ecostate Mapping project of 2019 at Santa Rita Experimental Range'

# top-level catalog definitions
catalog_id = 'Cyverse Remotely Sensed Imagery STAC Catalog'
catalog_description = 'This catalog includes all of the imagery assets the exist in Cyverse Data Store'


# create default datetime object for the collection - used when all items were collected on same date
default_datetime = datetime(year=2019, month=5, day=25, hour=12)

#If items were collected on different dates, then you should supply a csv that has the following columns:
#'Id', 'collection_date'
#df_collection_date = pandas.read_csv('/data-store/iplant/home/jgillan/STAC_drone/collection_date.csv')
df_collection_date = pandas.read_csv('/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019/srer_may2019.csv')


#Provider information
provider = Provider(name='Jeffrey Gillan',
                       description="A provider that supplies example geospatial data.",
                       roles=[ProviderRole.PRODUCER, ProviderRole.PROCESSOR],
                       url='https://www.gillanscience.com/')

provider_dict = provider.to_dict()

# email variables - DON'T CHANGE these unless you know what you're doing
contact_email = 'jgillan@arizona.edu'
smtp_host = '128.196.254.80'

In [11]:
#########Functions to extract the spatial resolution(ground sampline distance) from geotif imagery products \
##########These functions are using the 'rasterio' library.

# function to truncate spatial resolution to 3 decimal places
def trun_n_d(num,n):
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return '{0:.{1}f}'.format(num,n)
    i,p,d = num_s.partition('.')
    return '.'.join([i,(d+'0'*n)[:n]])

# function to get the spatial resolution of a raster
def spatial_resolution(raster):
    """extracts the XY Pixel Size"""
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [12]:
###########Function to calculate the bounding box and footprint of a geospatial raster dataset
######################

#This creates a function called 'get_bbox_and_footprint' for a raster we are calling 'dataset'
def get_bbox_and_footprint(dataset):

    # extract the bounding box of a raster using rasterio. '.bounds' is an attribute of 'rasterio.DatasetReader'
    #'bounds' returns the left, bottom, right, and top coordinates of a raster
    bounds = dataset.bounds
    
    #Transform the coordinate system of the raster bounds from it's orginial coordinate reference system \
    #to wgs84 which is also known as EPSG 4326. It uses the rasterio submodule 'rasterio.warp'
    bounds = rasterio.warp.transform_bounds(dataset.crs, 'EPSG:4326', 
                                            bounds.left, bounds.bottom, bounds.right, bounds.top)
   
    #The transformed bounds are then used to create a new rasterio.coords.BoundingBox object. \
    #The rasterio.coords.BoundingBox class is a convenient way to represent a bounding box \
    #with named attributes (left, bottom, right, and top) instead of using a tuple or a list. \
    #This makes the code more readable and easier to work with.
    bounds = rasterio.coords.BoundingBox(bounds[0], bounds[1], bounds[2], bounds[3])
    
    
    #The isinstance() function checks if the bounds variable is an instance of the \
    #rasterio.coords.BoundingBox class. If it is, it means that the bounds variable \
    #represents a bounding box with named attributes (left, bottom, right, and top).
    #If the bounds variable is an instance of rasterio.coords.BoundingBox, \
    #the code creates a list called bbox, which contains the bounding box coordinates \
    #in the following order: left, bottom, right, and top. This list is a more straightforward \
    #way to represent the bounding box as a sequence of coordinates.
    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        
    #If the bounds variable is not an instance of rasterio.coords.BoundingBox, \
    #the code assumes that it is a callable object (such as a function or a method) \
    #that returns the bounding box coordinates when called. In this case, the code \
    #calls the bounds() function and converts the returned bounding box coordinates \
    #to a list of floating-point numbers. This list is then assigned to the bbox variable.    
    else:
        bbox = [float(f) for f in bounds()]

    # create vertices and polygon from the bounding box coordinates. It uses the 'shapely.geometry' module \
    #from the shapely library. The output is the'footprint' variable which contains a shapely.geometry.Polygon object \
    #that represents the footprint of the raster dataset, based on its bounding box coordinates. \
    #This footprint can be used for tasks like spatial analysis, visualization, or overlaying with other geospatial data.
    footprint = Polygon([
        [bbox[0], bbox[1]],#left bottom
        [bbox[0], bbox[3]],#left top
        [bbox[2], bbox[3]],#right top
        [bbox[2], bbox[1]] #right bottom
    ])
    
    #Return the calculated bbox and the footprint as a dictionary using the mapping function \
    #(from the shapely.geometry module)
    return bbox, mapping(footprint)

In [ ]:
###This is a function to extract the timestamp of a raw drone image from the exif data embedded on the image

#import piexif

#def exif_to_timestamp(file_path):
    # EXIF tags to look for
   # EXIF_ORIGIN_TIMESTAMP = 36867         # Capture timestamp
   # EXIF_TIMESTAMP_OFFSET = 36881         # Timestamp UTC offset (general)
   # EXIF_ORIGIN_TIMESTAMP_OFFSET = 36881  # Capture timestamp UTC offset

   # cur_stamp, cur_offset = (None, None)

    # try to load EXIF data
   # exif_tags = piexif.load(file_path)
   # if not exif_tags or "Exif" not in exif_tags:
    #    return None
    
   # def convert_and_clean_tag(value):
        # internal helper function for handling EXIF tag values
    #    if not value:
    #        return None

        # Convert bytes to string
    #    if isinstance(value, bytes):
     #       value = value.decode('UTF-8').strip()
     #   else:
      #      value = value.strip()

        # Check for an empty string after stripping colons
     #   if value:
     #       if not value.replace(":", "").replace("+:", "").replace("-", "").strip():
      #          value = None

      #  return None if not value else value

    # Process the EXIF data
   # if EXIF_ORIGIN_TIMESTAMP in exif_tags:
      #  cur_stamp = convert_and_clean_tag(exif_tags[EXIF_ORIGIN_TIMESTAMP])
    #if not cur_stamp:
    #    return None

    #if EXIF_ORIGIN_TIMESTAMP_OFFSET in exif_tags:
     #   cur_offset = convert_and_clean_tag(exif_tags[EXIF_ORIGIN_TIMESTAMP_OFFSET])
    #if not cur_offset and EXIF_TIMESTAMP_OFFSET in exif_tags:
     #   cur_offset = convert_and_clean_tag(exif_tags[EXIF_TIMESTAMP_OFFSET])

    # Format the string to a timestamp and return the result
   # try:
    #    if not cur_offset:
     #       cur_ts = datetime.datetime.fromisoformat(cur_stamp)
     #   else:
      #      cur_offset = cur_offset.replace(":", "")
      #      cur_ts = datetime.datetime.fromisoformat(cur_stamp + cur_offset)
   # except Exception as ex:
   #     cur_ts = None
     #   print("Exception caught converting EXIF tag to timestamp: %s", str(ex))

  #  return cur_ts


In [13]:
####This starts the forloop to crawl through a folder, find imagery assets, and generate STAC json metadata files for each item
##################################################

# get a list of geospatial files (in this case located in Cyverse data store)
folder = Path(source_image_folder)
files = list(folder.rglob('*.tif'))#find files that end with .tif

items_dict = {}
all_items = []

# loop through each .tif item in the folder and do several things
for file in files:

    # open the individual file with rasterio
    ds = rasterio.open(file)

    # apply the function to get the bounding box (left, bottom, right, top) and make a footprint rectangle
    bbox, footprint = get_bbox_and_footprint(ds)

    # extract the spatial resolution (gsd) of the image product using the function 'spatial_resolution'.
    x_res,y_res = spatial_resolution(ds)

    # the ID (name) for each indivual file
    idx = file.stem
    
    
    #######The following 3 lines are for assigning the date of imagery collection to each of the item by matching IDs from a csv file
    
    #Within the forloop that we are in, this line looks at the 'Id' column of the csv file (imported into python as a pandas DataFrame).
    #It takes the Id name of the current geotiff file and looks for a match within the Id column within 'df_collection_date'. 
    #If it finds a matching ID, it returns the info for the entire row. IDs in the dataframe are stored as strings. 
    collection_time = df_collection_date[df_collection_date.Id == str(idx)]
    
    #From the matched row, this command will return the value within the 'collection_date' column as a 2D numpy array 
    dates = collection_time.collection_date.values
    
    #get the plot name of the item
    plot = collection_time['plot'].iloc[0]
    
    #Convert the date into a 'datetime' object (e.g., 2019-05-19)
    datess = datetime.strptime(dates[0], '%Y-%m-%d')

        
    ######Chris Schnaufer stuff 
    # get the timestamp to use - try to load capture date from file, otherwise use the start timestamp
    #file_datetime = exif_to_timestamp(str(file))
    #if not file_datetime:
    #file_datetime = default_datetime
       # print(f"Using the default timestamp for image {file}")
    
    #check if an item with this plot name already exists
    if plot in items_dict:
        #if it does, get that item
        item = items_dict[plot]
        
    #If this is an item with a new plot name 
    else:    
        # create a STAC item for each individual file 
        item = pystac.Item(id=plot,
                geometry=footprint,
                bbox=bbox,
                datetime=datess,
                stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
                                 'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
                 
                properties={'gsd': x_res,
                        'platform': platform,
                        'license': license,
                        'mission': items_mission_description,
                        'sci:doi': pub_doi,
                        'sci:citation': citation,
                        'providers': [provider_dict]}
        )
    items_dict[plot] = item
    
    # adding the map projection extension to each item. Otherwise, the projection info will not display 
    ProjectionExtension.ext(item).apply(ds.crs.to_epsg(),
                                        shape = ds.shape,
                                        bbox = bbox,
                                        geometry = footprint)
                                        #transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']
                                        #)
   
    #Add the asset link to the item and define the type of geospatial format it is
    item.add_asset(
        key=idx,
        asset=pystac.Asset(
            href=file.as_posix(), 
            media_type=pystac.MediaType.COG,
            roles='data'
            #extra_fields=asset_ext
        )
    )

    # add each STAC item to a list of all the items
    all_items.append(item)
    all_items = list(items_dict.values())

In [14]:
#######Create and describe a STAC Collection 
##########################

# the geographic extent of all the items added
item_extents = pystac.Extent.from_items(all_items)

# creating the collection
collection = pystac.Collection(id=collection_id,
                               description=collection_description,
                               extent=item_extents,
                               license=license)

# add all STAC item to the STAC collection
for item in all_items:
    collection.add_item(item)

In [16]:
# print the number of STAC items that were added to the STAC catalog    
print(len(list(collection.get_items())))

# describe the items in the STAC catalog
collection.describe()

0
* <Collection id=Drone_Imagery_Collection>


In [ ]:
##########Save the collection and its child items to output directory (ABSOLUTE LINKS FOR STAC API)

collection.normalize_hrefs(stac_output_directory)

collection.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)


In [ ]:
########Save the collection and it's child items with RELATIVE LINKS FOR STATIC CATALOG

#collection.normalize_hrefs(stac_output_directory)

#collection.make_all_asset_hrefs_relative()

#collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [ ]:
#########Combine all of the STAC item metadata into a single geojson and output to directory
####################

# Create an empty GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Iterate through all the items in the collection and convert them to GeoJSON Features
for item in collection.get_all_items():
    # Get the STAC Item as a dictionary
    item_dict = item.to_dict()

    # Convert the STAC Item to a GeoJSON Feature
    feature = {
        "type": "Feature",
        "collection": item_dict["collection"],
        "stac_version": item_dict["stac_version"],
        "stac_extensions": item_dict["stac_extensions"],
        "id": item_dict["id"],
        "geometry": item_dict["geometry"],
        "bbox": item_dict["bbox"],
        "properties": item_dict["properties"],
        "assets": item_dict["assets"]
    }

    # Add the GeoJSON Feature to the FeatureCollection
    geojson["features"].append(feature)


# Make sure the output_directory ends with a path separator
if not stac_output_directory.endswith("/"):
    stac_output_directory += "/"

output_file_path = stac_output_directory + "index.geojson"

# Write the GeoJSON FeatureCollection to a file
with open(output_file_path, "w") as f:
    json.dump(geojson, f, indent=4)    


#####We will now find a replace all internal datastore paths with the public Cyverse Data Store paths for each asset
#Define the string to find and the string to replace it with
string_to_find = "/data-store"
string_to_replace = "https://data.cyverse.org/dav-anon"

# Read the file into a string
with open(output_file_path, "r") as f:
    file_content = f.read()

# Perform the find-and-replace operation
file_content = file_content.replace(string_to_find, string_to_replace)

# Write the result back to the file
with open(output_file_path, "w") as f:
    f.write(file_content)


In [ ]:
# create a STAC catalog instance using pystac
#catalog = pystac.Catalog(
 #   id=catalog_id,
  #  description=catalog_description,
  #  stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
#)    

#add the collection to the catalog
#catalog.add_child(collection)

In [ ]:
# write the STAC catalog out
#catalog.normalize_hrefs(stac_output_directory)

#catalog.make_all_asset_hrefs_relative()
#catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

To make the saved catalog available, you will need to:
-  share the output folder to \"public\" using the [Discovery Environment](https://de.cyverse.org/) Data tab
- add your catalog to the CyVerse [master catalog](/iplant/home/jgillan/stac.cyverse.org/cyverse_stac_catalog/catalog.json)


In [ ]:
#%pip install python-irodsclient

The following will share the folder containing your catalog with the CyVerse `public` user using read-only permissions

You will be prompted to enter your CyVerse username and password. Be sure to use the *Enter* key for each prompt

In [ ]:
#import os
#from getpass import getpass
#from irods.access import iRODSAccess
#from irods.session import iRODSSession

#irods_username = input('Enter your CyVerse user name:')
#irods_password = getpass('Enter your CyVerse password:')

#sess = iRODSSession(host='data.cyverse.org', port=1247, user=irods_username, password=irods_password, zone='iplant')

#user = sess.users.get(sess.username, sess.zone)

# add the needed users for linking
#acl_path = '/' + os.path.join(*(stac_output_directory.split(os.path.sep)[2:]))
#acl = iRODSAccess('read', acl_path, 'anonymous', user.zone)
#sess.acls.set(acl, recursive=True)

#sess.cleanup()

#print("Folder updated")

**Almost Done!**

To have a *newly created* catalog added to the global CyVerse catalog, update `your_email` using your email address
and send an email using the following:

In [ ]:
#your_email = '<someone>@arizona.edu'

# Import smtplib for the actual sending function
#import smtplib

# Import the email modules we'll need
#from email.mime.text import MIMEText

#msg = MIMEText("Please add the following STAC catalog to the main CyVerse catalog: " + 
               #"https://data.cyverse.org/dav-anon/" + os.path.join(*(stac_output_directory.split(os.path.sep)[2:])) +
               #"/catalog.json")
#msg['Subject'] = 'Add new STAC catalog to main CyVerse catalog'
#msg['From'] = your_email
#msg['To'] = contact_email

# Send the message via our own SMTP server, but don't include the
# envelope header.
#smtp = smtplib.SMTP(smtp_host)
#smtp.sendmail(your_email, [contact_email], msg.as_string())
#smtp.quit()
